In [281]:
# import module
import requests
from bs4 import BeautifulSoup
import json
# UDF for get HTML code
# from URL


def get_html(Airline_code, Flight_number, Date, Month, Year):
    def getdata(url):
        r = requests.get(url)
        return r.text

    # url
    url = "https://www.flightstats.com/v2/flight-tracker/"+Airline_code + \
        "/"+Flight_number+"?year="+Year+"&month="+Month+"&date="+Date

    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')
    return(soup)

# Get Flight number
# from Html code


def flight_no(soup):
    Flight_no = ""

    # Find div tag with
    # unique class name
    for i in soup.find("div", class_="ticket__FlightNumberContainer-sc-1rrbl5o-4 exbpMf"):
        Flight_no = Flight_no + (i.get_text()) + " "
    return Flight_no

# Get Airport name
# from HTML code


def airport(soup):
    Airport_name = []
    # Find div tag with
    # unique class name
    for i in soup.find_all(class_='text-helper__TextHelper-sc-8bko4a-0 cHdMkI'):
        Airport_name.append(i.get_text())
    return Airport_name

# get status
# from HTML code


def status(soup, flight_no):
  for i in soup.find_all("script"):
    if '__NEXT_DATA__' in i.get_text():
      result = i

  data = json.loads(result.string.split('{"props":')[1].split(',"page":')[0])
  status = data['initialState']['flightTracker']['flight']['status']
  departure = data['initialState']['flightTracker']['flight']['departureAirport']
  arrival = data['initialState']['flightTracker']['flight']['arrivalAirport']

  flight_details = {
    'flight': flight_no,
    'status': (status['statusDescription'], status['delay']['departure']['minutes'], status['delay']['arrival']['minutes']),
    'source': (departure['iata'], departure['name'], departure['gate'], departure['times']['scheduled']['time'], departure['times']['scheduled']['ampm'], departure['times']['estimatedActual']['time'], departure['times']['estimatedActual']['ampm']),
    'destination': (arrival['iata'], arrival['name'], arrival['gate'], arrival['times']['scheduled']['time'], arrival['times']['scheduled']['ampm'], arrival['times']['estimatedActual']['time'], arrival['times']['estimatedActual']['ampm'])
  }
  return flight_details


In [284]:
# Driver code
if __name__ == '__main__':
    # Input Data from geek
    Airline_code = 'AA'
    Flight_number = '3'
    Date = '26'
    Month = '5'
    Year = '2024'

    # Calling the get_html
    # with argument
    # function calling
    soup = get_html(Airline_code, Flight_number, Date, Month, Year)
    flight_num = flight_no(soup)
    print("Flight number : ", )
    Airport_list = airport(soup)
    result = status(soup, flight_num)
    print(result)

Flight number : 
{'flight': 'AA 3 American Airlines ', 'status': ('On time', 0, 0), 'source': ('JFK', 'John F. Kennedy International Airport', '1', '11:00', 'AM', '10:56', 'AM'), 'destination': ('LAX', 'Los Angeles International Airport', '47A', '2:08', 'PM', '2:05', 'PM')}
